<img src="CBS logo.png" alt="University Logo" style="float: left; margin-right: 10px; width: 400px;">


<h1 style="text-align: center;">- Machine Learning Assigment 3 -</h1>


***

**Students names and numbers:**
- Charalampos Karatzas, 176175

***

## Question 1 [Perceptron]

*Suppose we want to train a perceptron (refer to Figure 1) with weights w1 and w2 and a fixed bias
b = −1 Sketch the constraints in weight space corresponding to the following training cases.*

In this task I will use the heaviside stepfunctions: 
$$
\text{heaviside}(z) = 
\begin{cases}
0 & \text{if } z < 0 \\
1 & \text{if } z \ge 0
\end{cases}
$$

$z = \mathbf{w} \cdot \mathbf{x}^\top + b$.


For $t = 1$, $\mathbf{x} = (1, -1)$:

$$
z = \mathbf{w} \mathbf{x}^\top + b \ge 0 \Rightarrow w_1 - w_2 - 1 \ge 0
$$

For $t = 0$, $\mathbf{x} = (-1, -1)$:

$$
z = \mathbf{w} \mathbf{x}^\top + b < 0 \Rightarrow w_1 - w_2 - 1 < 0
$$


<img src="Task1.png" alt="Task 1" style="float: left; margin-right: 10px; width: 600px;">

***

## Question 2 [Neural Networks]

*The following is a network (refer to Figure 2) of linear neurons, that is, neurons whose output is
identical to their net input. The numbers in the circles indicate the outcome of a neuron, and the
numbers at connections indicate the value of the corresponding weight.*
1. Compute the output of the hidden layer and the output-layer neurons for the given input (0.5,1) and enter those values into the corresponding circles.
2. What is the network output for the input (1, 2) (i.e., the left input neuron having the value one
and the right one having the value 2)? Do you have to do all the network computations once
again to answer this question? Explain why you do or do not have to do this.

#### 2.1

To compute the output of the hidder layer neurons we use this:
$$
h_j = \sum_i w_{j,i} x_i + b_j
$$

To compute the output of the output-layer neuros we use this:
$$
y_k = \sum_j v_{k,j} h_j + c_k
$$

*** 

$$
h_1 = -1 \times 0.5 + 1 \times 0 = -0.5
$$

$$
h_2 = 0.5 \times 2 + 1 \times 1 = 2
$$

$$
h_3 = 0.5 \times (-2) + 1 \times 1 = 0
$$

$$
y_1 = 2 \times (-0.5) + 2 \times (-0.5) + 0 \times 1 = -2
$$

$$
y_2 = -2 \times (-0.5) + 1 \times 2 + 0 \times 0.5 = 3
$$




<img src="Task2_1.png" alt="Task2_1.png" style="float: left; margin-right: 10px; width: 500px;">

***

#### 2.2

<img src="Task2_2.png" alt="Task2_1.png" style="float: left; margin-right: 10px; width: 500px;">

Its not necessary to compute everything again since it's easy to understand that the input vector is the first one multiplied by 2. So since the network is linear we can go straight away to compute the output which is the output of the first task multiplied by 2.

***

## Question 3 [Coding]

*In this question, you will use the Fashion MNIST dataset and build a custom model using custom
training loops (using a different optimizer with a different learning rate for the upper layers
and the lower layers) to tackle image classification in the Fashion MNIST dataset.
For example, you can use Sequential model from keras to build your custom model. Please note
that Sequential model is suitable for a simple stack of layers with the restriction that each layer can
only support exactly one input tensor and one output tensor. Alternatively, you can also use Keras
Functional API, which allows you to create models that are more flexible than the models created
using Sequential model. Some hints are as follows:*

1. Only use five epochs and 32 as batch size.
2. Only use softmax and ReLU activation functions.
3. Use SGD as the lower optimizer with the learning rate of 1e-4 and Nadam as upper optimizer
with a learning rate as 1e-3.
4. Use Nadam optimizer from Keras and also use sparse categorical cross entropy as a loss function.
5. Display the mean training loss and the mean accuracy over each epoch (updated at each itera-
tion). Also display, validation loss, and accuracy at the end of each epoch.

### Import the Fashion MNIST dataset

In [5]:
import sys
import sklearn
import os #To hide out some info tensorflow gives for my pc
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
from tensorflow import keras
import numpy as np

# to make this notebook ’s output stable across runs
np.random.seed (42)
tf.random.set_seed (42)

#Load the dataset
( X_train_full , y_train_full ) , ( X_test , y_test ) = keras.datasets.fashion_mnist.load_data ()

#Normalize the values to [0,1]
X_train_full = X_train_full.astype( np . float32 ) / 255.
X_valid , X_train = X_train_full [:5000] , X_train_full [5000:]
y_valid , y_train = y_train_full [:5000] , y_train_full [5000:]
X_test = X_test.astype(np.float32 ) / 255


***

### Model Construction

**We are going to build a simple Sequential model**

- A flatten layer to convert the 28x28 images into a 1D vector
- A dense hidden layer with 128 neurons and ReLu activation ( for the lower part )
- An ouutput dense layer with 10 neurons and softmax activation ( for the upper layer )

In [6]:
#We are going to use as recommended the Keras Sequential API

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28,28)),
    keras.layers.Dense(128, activation = 'relu'), #For the lower layer
    keras.layers.Dense(10,activation = 'softmax') #For the upper layer
])

***

### Defining Loss Function and Metrics

- Use sparse categorical cross entropy as the loss function
- Define metrics to monitor both training and validation performance

In [20]:
#Loss function
loss_fun = keras.losses.SparseCategoricalCrossentropy()

#Defining training metrics
train_loss_metric = keras.metrics.Mean()
train_accuracy_metric = keras.metrics.SparseCategoricalAccuracy()

#Define validation metrics
val_loss_metric = keras.metrics.Mean()
val_accuracy_metric = keras.metrics.SparseCategoricalAccuracy()

### Optimizers and Variable Partitioning

- SGD with learning rate 1e-4 for the lower layer
- Nadam with learning rate 1e-3 for the upper layer
- Partition the model's trainable variables into two groups based on layers

In [21]:
#Create optimizers with the specified learning rates 
optimizer_lower = keras.optimizers.legacy.SGD(learning_rate = 1e-4)
optimizer_upper = keras.optimizers.legacy.Nadam(learning_rate = 1e-3) #legacy because the simple optimizers.SGD said it's too slow for my M2 chip and it reccomended legacy.SGD

#Partition variables
lower_vars = model.layers[1].trainable_variables
upper_vars = model.layers[2].trainable_variables

## We made two Custom Functions. The second is based on the book Hands-on Machine Learning with Scikit-Learn,Keras & TensorFlow

### Custom Training Loop #1

- Create a Dataset for training and validation with a batch size of 32
- For each taining batch:
    - Use tf.GradientTape to compute loss and gradients
    - Split the gradients into two groups(for lower and upper layers)
    - Apply the corresponding optimizer update for each group
    - Update training metrics and print running resukts every 100 steps
- After each epoch, evaluate the model on the validation set and print the validation loss and accuarcy.

In [23]:
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size = 10000).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(batch_size)

epochs = 5

#Custom traiing loop
for epoch in range(epochs):
    print(f"\nEpoch {epoch+1}/{epochs}")
    train_loss_metric.reset_states()
    train_accuarcy_metric.reset_states()

    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fun(y_batch, logits)

        grads = tape.gradient(loss_value, model.trainable_variables)

        num_lower = len(lower_vars)
        grads_lower = grads[:num_lower]
        grads_upper = grads[num_lower:]

        optimizer_lower.apply_gradients(zip(grads_lower, lower_vars))
        optimizer_upper.apply_gradients(zip(grads_upper, upper_vars))
        
    
        train_loss_metric.update_state(loss_value)
        train_accuracy_metric.update_state(y_batch, logits)
        
        if step % 100 == 0:
            print(f"Step {step}: Mean loss = {train_loss_metric.result().numpy():.4f}, Accuracy = {train_accuracy_metric.result().numpy():.4f}")
    
    val_loss_metric.reset_states()
    val_accuracy_metric.reset_states()
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        val_loss = loss_fun(y_batch_val, val_logits)
        val_loss_metric.update_state(val_loss)
        val_accuracy_metric.update_state(y_batch_val, val_logits)
    
    print(f"Epoch {epoch+1}: Training loss = {train_loss_metric.result().numpy():.4f}, Training accuracy = {train_accuracy_metric.result().numpy():.4f}")
    print(f"Epoch {epoch+1}: Validation loss = {val_loss_metric.result().numpy():.4f}, Validation accuracy = {val_accuracy_metric.result().numpy():.4f}")


Epoch 1/5
Step 0: Mean loss = 0.4263, Accuracy = 0.7929
Step 100: Mean loss = 0.5795, Accuracy = 0.7929
Step 200: Mean loss = 0.5699, Accuracy = 0.7931
Step 300: Mean loss = 0.5702, Accuracy = 0.7934
Step 400: Mean loss = 0.5652, Accuracy = 0.7947
Step 500: Mean loss = 0.5639, Accuracy = 0.7949
Step 600: Mean loss = 0.5641, Accuracy = 0.7952
Step 700: Mean loss = 0.5645, Accuracy = 0.7960
Step 800: Mean loss = 0.5613, Accuracy = 0.7970
Step 900: Mean loss = 0.5613, Accuracy = 0.7974
Step 1000: Mean loss = 0.5598, Accuracy = 0.7980
Step 1100: Mean loss = 0.5578, Accuracy = 0.7986
Step 1200: Mean loss = 0.5560, Accuracy = 0.7990
Step 1300: Mean loss = 0.5542, Accuracy = 0.7997
Step 1400: Mean loss = 0.5527, Accuracy = 0.8000
Step 1500: Mean loss = 0.5517, Accuracy = 0.8003
Step 1600: Mean loss = 0.5526, Accuracy = 0.8002
Step 1700: Mean loss = 0.5513, Accuracy = 0.8008
Epoch 1: Training loss = 0.5512, Training accuracy = 0.8009
Epoch 1: Validation loss = 0.5152, Validation accuracy = 0.

***

### Custom Function #2

In [3]:
def random_batch(X, y, batch_size=32):
    idx = np.random.randint(len(X), size=batch_size)
    return X[idx], y[idx]
    
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())
                         for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

                          

In [4]:

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_train_scaled = X_train

model = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])
lower_vars = model.layers[1].trainable_variables
upper_vars = model.layers[2].trainable_variables

n_epochs = 5
batch_size = 32
n_steps = len(X_train) // batch_size
optimizer_upper = keras.optimizers.legacy.Nadam(learning_rate=1e-3)
optimizer_lower = keras.optimizers.legacy.SGD(learning_rate=1e-4)
loss_fn = keras.losses.sparse_categorical_crossentropy
mean_loss = keras.metrics.Mean()
metrics = [keras.metrics.SparseCategoricalAccuracy()]

val_loss = keras.metrics.Mean(name="val_loss")
val_accuracy = keras.metrics.SparseCategoricalAccuracy(name="val_accuracy")

for epoch in range(1, n_epochs + 1): 
    print("Epoch {}/{}".format(epoch, n_epochs))
    for step in range(1, n_steps + 1):
        X_batch, y_batch = random_batch(X_train_scaled, y_train)
        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        gradients = tape.gradient(loss, model.trainable_variables)
        num_lower = len(lower_vars)
        grads_lower = gradients[:num_lower]
        grads_upper = gradients[num_lower:]
        optimizer_lower.apply_gradients(zip(grads_lower, lower_vars))
        optimizer_upper.apply_gradients(zip(grads_upper, upper_vars))
        mean_loss(loss)
        for metric in metrics:
            metric(y_batch, y_pred)
        print_status_bar(step * batch_size, len(y_train), mean_loss, metrics)
    print_status_bar(len(y_train), len(y_train), mean_loss, metrics)
    n_val_steps = len(X_valid) // batch_size
    for v_step in range(1, n_val_steps + 1):
        start = (v_step - 1) * batch_size
        end = v_step * batch_size
        X_val_batch = X_valid[start:end]
        y_val_batch = y_valid[start:end]
        y_val_pred = model(X_val_batch, training=False)
        v_loss = tf.reduce_mean(loss_fn(y_val_batch, y_val_pred))
        val_loss(v_loss)
        val_accuracy(y_val_batch, y_val_pred)
    print("Validation - Loss: {:.4f}, Accuracy: {:.4f}".format(val_loss.result().numpy(), val_accuracy.result().numpy()))
    for metric in [mean_loss] + metrics + [val_loss, val_accuracy]:
        metric.reset_states()


Epoch 1/5
55000/55000 - mean: 0.9891 - sparse_categorical_accuracy: 0.7066
Validation - Loss: 0.6575, Accuracy: 0.7929
Epoch 2/5
55000/55000 - mean: 0.6217 - sparse_categorical_accuracy: 0.7901
Validation - Loss: 0.5607, Accuracy: 0.8171
Epoch 3/5
55000/55000 - mean: 0.5541 - sparse_categorical_accuracy: 0.8078
Validation - Loss: 0.5246, Accuracy: 0.8241
Epoch 4/5
55000/55000 - mean: 0.5273 - sparse_categorical_accuracy: 0.8132
Validation - Loss: 0.5077, Accuracy: 0.8321
Epoch 5/5
55000/55000 - mean: 0.5078 - sparse_categorical_accuracy: 0.8217
Validation - Loss: 0.4928, Accuracy: 0.8365
